# Filter Change MTPtg/Rotator Integration Test

This notebook relates to Jira test case: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T1595

This notebook performs a filter change scenario integration test between the Maint Telescope pointing component (MTPtg) and the Rotator with the Camera Cable Wrap tracking the Rotator. It includes enough boilerplate to allow the test to run at any time by getting current time information from the pointing and computing appropriate coordinates to slew.

In [1]:
import logging
import yaml

import os
import sqlite3
import pandas as pd

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio
from lsst.ts import salobj
from lsst.ts.idl.enums import MTPtg

from astropy.utils import iers
iers.conf.auto_download = False

from datetime import datetime 

In [2]:
test_message = "Filter Change MTPtg_Rotator_CCW Integration Test - LVV-T1595"
now = datetime.now()
print(test_message, now)

Filter Change MTPtg_Rotator_CCW Integration Test 2021-06-18 18:08:55.687819


In [3]:
con = sqlite3.connect(os.path.expanduser("~/develop/one_filt_v1.4_0yrs_1.db"))

OperationalError: unable to open database file

In [4]:
df = pd.read_sql_query("SELECT * from SummaryAllProps", con)

NameError: name 'con' is not defined

In [5]:
d = salobj.Domain()

In [7]:
script = salobj.Controller("Script", index=1)
rot = salobj.Remote(d, "MTRotator")
mtptg = salobj.Remote(d, "MTPtg")
mtm = salobj.Remote(d, "MTMount")

In [8]:
await asyncio.gather(rot.start_task,
                     mtptg.start_task,
                     script.start_task,
                     mtm.start_task)

rotation DDS read queue is full (100 elements); data may be lost
motors DDS read queue is full (100 elements); data may be lost
electrical DDS read queue is full (100 elements); data may be lost
ccwFollowingError DDS read queue is full (100 elements); data may be lost
timeAndDate DDS read queue is filling: 12 of 100 elements
mountStatus DDS read queue is filling: 14 of 100 elements


[None, None, None, None]

mountPosition DDS read queue is filling: 14 of 100 elements
currentTargetStatus DDS read queue is filling: 15 of 100 elements


In [14]:
await asyncio.sleep(1.)
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)

[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [15]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

[<State.ENABLED: 2>]

In [ ]:
await mtm.cmd_enterControl.start(timeout=10.)
await mtm.cmd_start.start(timeout=10.)
await mtm.cmd_enable.start(timeout=10.)

In [11]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [12]:
def parallactic_angle(location, lst, target):
    """
    Calculate the parallactic angle.
    Parameters
    ----------
    time : `~astropy.time.Time`
        Observation time.
    target : `~astroplan.FixedTarget` or `~astropy.coordinates.SkyCoord` or list
        Target celestial object(s).
    grid_times_targets: bool
        If True, the target object will have extra dimensions packed onto the end,
        so that calculations with M targets and N times will return an (M, N)
        shaped result. Otherwise, we rely on broadcasting the shapes together
        using standard numpy rules.
    Returns
    -------
    `~astropy.coordinates.Angle`
        Parallactic angle.
    Notes
    -----
    The parallactic angle is the angle between the great circle that
    intersects a celestial object and the zenith, and the object's hour
    circle [1]_.
    .. [1] https://en.wikipedia.org/wiki/Parallactic_angle
    """
    # Eqn (14.1) of Meeus' Astronomical Algorithms
    H = (lst - target.ra).radian
    q = np.arctan2(np.sin(H),
                   (np.tan(location.lat.radian) *
                    np.cos(target.dec.radian) -
                    np.sin(target.dec.radian)*np.cos(H)))*u.rad
    return Angle(q)

# Move to 0 deg Starting Position

In [18]:

now = datetime.now()
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"START - {test_message} - Move to 0 deg Starting Position - {now} UTC")

now = datetime.now()
print("Move to 0.0 deg starting position", now)

alt = 45. * u.deg
az = 0. * u.deg
rot_tel = Angle(0, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.mjd, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

# Calculating the other parameters     
rot_pa = rot_tel
print(rot_pa)

await rot.cmd_trackStart.start(timeout=30.)
    
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    #targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=MTPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    trackId=9999,
    rotAngle=15.0,
    #rotPA=rot_pa.deg,
    #rotFrame=ATPtg.RotFrame.TARGET,
    rotStartFrame=MTPtg.RotFrame.FIXED,
    rotTrackFrame=MTPtg.RotFrame.FIXED,
    rotMode=MTPtg.RotMode.FIELD,
    azWrapStrategy=2,
    timeOnTarget=30,
    timeout=10
)

print("Waiting 30s")
await asyncio.sleep(30.)

Move to 0.0 deg starting position 2021-06-18 21:39:35.673684
Time error=-0.00 sec
0d00m00s
Waiting 30s


In [ ]:
await mtptg.cmd_stopTracking.start(timeout=10.)

In [ ]:
await mtm.cmd_stop.start(timeout=10.)

In [17]:
await rot.cmd_stop.start(timeout=10.)

# Start Test

In [19]:
now = datetime.now()
print("Start Test", now)
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"START - {test_message} - {now} UTC")

Start Test 2021-06-18 21:40:05.744073


True

# Perform First Track

In [20]:
now = datetime.now()
print("Perform First Track", now)
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"START - {test_message} - Perform First Track - {now} UTC")

field = 0
target_name = df["note"][field]
alt = df['altitude'][field] * u.deg
az = df['azimuth'][field] * u.deg
rot_tel = Angle(df["rotTelPos"][field]*u.deg)
para_ang = Angle(df["paraAngle"][field]*u.deg)
rot_pa = para_ang-180*u.deg+rot_tel.wrap_at('180d')
visit_time = df['visitTime'][field]  # how long the visit lasted. Usefull so you can wait between visits
print(f"Alt: {alt}\nAz: {az}\nRotTel: {rot_tel}")

#Here is a small trick to get the ra before transit. Get `timeAndDate` from the pointing component, then use `RA = lst - delta`.
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.mjd, format="mjd", scale="tai")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, obstime=curr_time_atptg.tai, location=location)
cmd_radec = cmd_elaz.transform_to(ICRS) # This is the RA/Dec of the target

print(f"RA: {cmd_radec.ra}\nDec: {cmd_radec.dec}\nRotPA: {rot_pa}")

await rot.cmd_trackStart.start(timeout=30.)

ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    #targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=MTPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    trackId=9999,
    rotAngle=15.0,
    #rotPA=rot_pa.deg,
    #rotFrame=ATPtg.RotFrame.TARGET,
    rotStartFrame=MTPtg.RotFrame.FIXED,
    rotTrackFrame=MTPtg.RotFrame.FIXED,
    rotMode=MTPtg.RotMode.FIELD,
    azWrapStrategy=2,
    timeOnTarget=30,
    timeout=10
)

print("Waiting", visit_time, "s")
await asyncio.sleep(visit_time)

Perform First Track 2021-06-18 21:44:20.226894


NameError: name 'df' is not defined

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=10.)
await mtm.cmd_stop.start(timeout=10.)

In [ ]:
await rot.cmd_stop.start(timeout=10.)

# Perform Filter Change - Move Command

# Perform Filter Change - Track Command

In [21]:
now = datetime.now()
print("Perform First Track", now)
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"START - {test_message} - Perform Filter Change incl. Track Command - {now} UTC")
now = datetime.now()
print("Move to 0.0 deg for filter change",now)

field = 1
alt = df['altitude'][field] * u.deg
az = df['azimuth'][field] * u.deg
rot_tel = Angle(0, unit=u.deg)

target_name="filter change"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.mjd, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

#Here is a small trick to get the ra before transit. Get `timeAndDate` from the pointing component, then use `RA = lst - delta`.
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.mjd, format="mjd", scale="tai")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, obstime=curr_time_atptg.tai, location=location)
cmd_radec = cmd_elaz.transform_to(ICRS) # This is the RA/Dec of the target

# Calculating the other parameters     
rot_pa = rot_tel
print(rot_pa)

await rot.cmd_trackStart.start(timeout=30.)

ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    #targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=MTPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    trackId=9999,
    rotAngle=15.0,
    #rotPA=rot_pa.deg,
    #rotFrame=ATPtg.RotFrame.TARGET,
    rotStartFrame=MTPtg.RotFrame.FIXED,
    rotTrackFrame=MTPtg.RotFrame.FIXED,
    rotMode=MTPtg.RotMode.FIELD,
    azWrapStrategy=2,
    timeOnTarget=30,
    timeout=10
)

print("Waiting 30s to move to 0 deg")
await asyncio.sleep(30.)

Move to 0.0 deg for filter change 2021-06-18 21:45:20.418051


NameError: name 'df' is not defined

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=10.)
await mtm.cmd_stop.start(timeout=10.)

In [ ]:
await rot.cmd_stop.start(timeout=10.)

In [ ]:
print("Rotator in position and waiting 90s")
await asyncio.sleep(90.)

# Perform Second Track

In [22]:
now = datetime.now()
print("Perform Second Track", now)
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"START - {test_message} - Perform Second Track- {now} UTC")

target_name = df["note"][field]
#alt = df['altitude'][field] * u.deg
#az = df['azimuth'][field] * u.deg
rot_tel = Angle(df["rotTelPos"][field]*u.deg)
para_ang = Angle(df["paraAngle"][field]*u.deg)
rot_pa = para_ang-180*u.deg+rot_tel.wrap_at('180d')
visit_time = df['visitTime'][field]  # how long the visit lasted. Usefull so you can wait between visits
print(f"Alt: {alt}\nAz: {az}\nRotTel: {rot_tel}")

await rot.cmd_trackStart.start(timeout=30.)

ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    #targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=MTPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    trackId=9999,
    rotAngle=15.0,
    #rotPA=rot_pa.deg,
    #rotFrame=ATPtg.RotFrame.TARGET,
    rotStartFrame=MTPtg.RotFrame.FIXED,
    rotTrackFrame=MTPtg.RotFrame.FIXED,
    rotMode=MTPtg.RotMode.FIELD,
    azWrapStrategy=2,
    timeOnTarget=30,
    timeout=10
)

print("Waiting", visit_time, "s")
await asyncio.sleep(visit_time)

Perform Second Track 2021-06-18 21:52:50.789953


NameError: name 'df' is not defined

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=10.)
await mtm.cmd_stop.start(timeout=10.)
await rot.cmd_stop.start(timeout=10.)

# Move to Zero - Move Command

# Move to Park Position - Track Command

In [24]:
now = datetime.now()
print("Move to 0.0 deg starting position", now)
script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"START - {test_message} - Move to Park Position incl. Track Command - {now} UTC")

alt = 60. * u.deg
az = 0. * u.deg
rot_tel = Angle(0, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.mjd, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

# Calculating the other parameters     
rot_pa = rot_tel
print(rot_pa)

await rot.cmd_trackStart.start(timeout=30.)

ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    #targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=MTPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    trackId=9999,
    rotAngle=15.0,
    #rotPA=rot_pa.deg,
    #rotFrame=ATPtg.RotFrame.TARGET,
    rotStartFrame=MTPtg.RotFrame.FIXED,
    rotTrackFrame=MTPtg.RotFrame.FIXED,
    rotMode=MTPtg.RotMode.FIELD,
    azWrapStrategy=2,
    timeOnTarget=30,
    timeout=10
)

print("Waiting 30s")
await asyncio.sleep(30.)

Move to 0.0 deg starting position 2021-06-18 21:54:46.398335
Time error=-0.00 sec
0d00m00s


AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=2030597046, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Low-level controller in substate 2.0 instead of <EnabledSubstate.STATIONARY: 0>')

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=10.)
await mtm.cmd_stop.start(timeout=10.)
await rot.cmd_stop.start(timeout=10.)

# Test Complete

In [25]:
now = datetime.now()
print("Test Complete",now)

script.evt_logMessage.set_put(level=logging.INFO+1,
                              message=f"END - {test_message} - {now} UTC")



Test Complete 2021-06-18 22:18:39.211177


True

# Additional Error Handling Commands

# Bring the Rotator and CCW back to enabled state

In [ ]:
await mtm.cmd_clearerror.start(timeout=30)

Wait for override off

In [ ]:
await rot.cmd_clearError.start(timeout=30)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await mtm.cmd_enterControl.start(timeout=30.)
await mtm.cmd_start.start(timeout=30.)
await mtm.cmd_enable.start(timeout=30.)

# Move to sync position with CCW

In [ ]:
print("Move to X.X deg starting position")

await rot.cmd_positionSet.set_start(angle=19.04,timeout=30.)

await rot.cmd_move.start(timeout=30.)

In [ ]:
await rot.cmd_clearError.start(timeout=10)

In [ ]:
await rot.cmd_exitControl.start(timeout=10.)

In [ ]:
await rot.cmd_enterControl.start(timeout=10.)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await rot.cmd_stop.start(timeout=10.)

In [ ]:
await mtm.cmd_clearerror.start(timeout=10)

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=10)
await mtm.cmd_stop.start(timeout=10)

In [ ]:
await mtm.cmd_stop.start(timeout=10)

Bring CCW Down

In [ ]:
await mtm.cmd_disable.start(timeout=30.)
await mtm.cmd_standby.start(timeout=30.)
await mtm.cmd_exitControl.start(timeout=30.)

Wait for override off

In [ ]:
await mtm.cmd_enterControl.start(timeout=30.)
await mtm.cmd_start.start(timeout=30.)
await mtm.cmd_enable.start(timeout=30.)

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=30)

In [ ]:
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)